<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---

# **World Happiness Model Playground Deployment & Competition Creation**

# Objective: Predict World Happiness Rankings 

What makes the citizens of one country more happy than the citizens of other countries?  Do variables measuring perceptions of corruption, GDP, maintaining a healthy lifestyle, or social support associate with a country's happiness ranking?  

Let's use the United Nation's World Happiness Rankings country level data to experiment with models that predict happiness rankings well.


---

**Data**: 2019 World Happiness Survey Rankings

**Features**
*   Country or region
*   GDP per capita
*   Social support
*   Healthy life expectancy
*   Freedom to make life choices
*   Generosity
*   Perceptions of corruption
*   Terrorist attacks

**Target**
*   Happiness_level (Very High = Top 20% and Very Low = Bottom 20%)

Source: https://worldhappiness.report/

## **Credential Configuration**

In order to deploy an AI Model Share Model Playground, you will need a credentials text file. 

Generating your credentials file requires two sets of information: 
1. Your AI Model Share username and password (create them [HERE](https://www.modelshare.org/login)). 
2. Your AWS (Amazon Web Services) access keys (follow the tutorial [HERE](https://docs.aws.amazon.com/general/latest/gr/aws-sec-cred-types.html)). 

You only need to generate your credentials file once. After running the configure function below, save the outputted file for all your future Model Playground deployments and competition submissions. 

*Note: Handle your credentials file with the same level of security you handle your passwords. Do not share your file with anyone, send via email, or upload to Github.*

In [ ]:
# install aimodelshare library
! pip install aimodelshare --upgrade

In [ ]:
# Generate credentials file (skip this step if you have a file already)
import aimodelshare as ai 
from aimodelshare.aws import configure_credentials 

configure_credentials()

AI Modelshare Username:··········
AI Modelshare Password:··········
AWS_ACCESS_KEY_ID:··········
AWS_SECRET_ACCESS_KEY:··········
AWS_REGION:··········
Configuration successful. New credentials file saved as 'credentials.txt'


# **Part 1: Prep Data, Preprocessor, & Model**

In [ ]:
from aimodelshare.aws import set_credentials
set_credentials(credential_file = "credentials.txt", type="deploy_model")

AI Model Share login credentials set successfully.
AWS credentials set successfully.


In [ ]:
# Get deployment data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/world_happiness_deployment_data-repository:latest')


Data downloaded successfully.


In [ ]:
## Load Data 
import pandas as pd

data=pd.read_csv("world_happiness_deployment_data/World_happinessdata2019.csv")
data.head()

,Happiness_level,Country or region,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,name,region,sub-region,Terrorist_attacks
0,Very High,Finland,1.340,1.587,0.986,0.596,0.153,0.393,Finland,Europe,Northern Europe,57.333333
1,Very High,Denmark,1.383,1.573,0.996,0.592,0.252,0.410,Denmark,Europe,Northern Europe,2.000000
2,Very High,Norway,1.488,1.582,1.028,0.603,0.271,0.341,Norway,Europe,Northern Europe,1.000000
3,Very High,Iceland,1.380,1.624,1.026,0.591,0.354,0.118,Iceland,Europe,Northern Europe,1.000000
4,Very High,Netherlands,1.396,1.522,0.999,0.557,0.322,0.298,Netherlands,Europe,Western Europe,1.000000


In [ ]:
# Create X and Y data objects
X = data.drop(['Happiness_level'], axis=1)
y = data['Happiness_level']

X.shape, y.shape

((156, 11), (156,))

In [ ]:
# Set up training and test data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_train.columns.tolist())

(104, 11)
(104,)
['Country or region', 'GDP per capita', 'Social support', 'Healthy life expectancy', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption', 'name', 'region', 'sub-region', 'Terrorist_attacks']


### **Define Preprocessor** <br>
Preprocess data using sklearn's Column Transformer

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# We create the preprocessing pipelines for both numeric and categorical data.
X = X.drop(['Country or region', 'name'], axis=1)

numeric_features=X.columns.tolist()
numeric_features.remove('region')
numeric_features.remove('sub-region')

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['region', 'sub-region']

#Replacing missing values with Modal value and then one hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# final preprocessor object set up with ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

#Fit your preprocessor object
preprocess=preprocessor.fit(X_train) 

In [ ]:
# Write function to transform data with preprocessor

def preprocessor(data):
    data.drop(['Country or region', 'name'], axis=1)
    preprocessed_data=preprocess.transform(data)
    return preprocessed_data

In [ ]:
# Check shape for keras input:
preprocessor(X_train).shape # pretty small dataset

(104, 26)

In [ ]:
# preserve y_train labels
y_train_labels = y_train
y_train = pd.get_dummies(y_train)

In [ ]:
#Save Example Data 
example_data = X_train[50:55]
example_data

,Country or region,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,name,region,sub-region,Terrorist_attacks
7,New Zealand,1.303,1.557,1.026,0.585,0.330,0.380,New Zealand,Oceania,Australia and New Zealand,57.333333
43,Slovenia,1.258,1.523,0.953,0.564,0.144,0.057,Slovenia,Europe,Southern Europe,125.611111
70,Moldova,0.685,1.328,0.739,0.245,0.181,0.000,Moldova,Europe,Eastern Europe,2.000000
113,Niger,0.138,0.774,0.366,0.318,0.188,0.102,Niger,Africa,Sub-Saharan Africa,24.000000
91,Indonesia,0.931,1.203,0.660,0.491,0.498,0.028,Indonesia,Asia,South-eastern Asia,84.000000


### **Train Placeholder Model** <br>
Neural network with Keras

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

feature_count=preprocessor(X_train).shape[1] #count features in input data

model = Sequential()
model.add(Dense(64, input_dim=feature_count, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(5, activation='softmax')) 
                                            
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model.fit(preprocessor(X_train), y_train, 
               batch_size = 60, 
               epochs = 300, validation_split=0.35)  

**Save Preprocessor & Model to local files**

In [ ]:
#export preprocessor
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"")

Your preprocessor is now saved to 'preprocessor.zip'


In [ ]:
# Test Preprocessor
prep = ai.import_preprocessor("preprocessor.zip")
prep(example_data).shape

(5, 26)

In [ ]:
# Save keras model to ONNX file 

from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

# **Part 2: Deploy Model**

In [ ]:
#Set up arguments for Model Playground deployment
import pandas as pd 

model_filepath="model.onnx"
preprocessor_filepath="preprocessor.zip"
exampledata = example_data

In [ ]:
#Deploy API
from aimodelshare import ModelPlayground

#Instantiate ModelPlayground() Class
myplayground=ModelPlayground(model_type="tabular", classification=True, private=False)

# Create Model Playground (generates live rest api and web-app for your model/preprocessor)

myplayground.deploy(model_filepath, preprocessor_filepath, y_train, exampledata) 

We need some information about your model before we can build your REST API and interactive Model Playground.
   
Model Name (for AI Model Share Website):World Happiness Classifier
Model Description (Explain what your model does and 
 why end-users would find your model useful):What makes the citizens of one country more happy than the citizens of other countries? Do variables measuring perceptions of corruption, GDP, maintaining a healthy lifestyle, or social support associate with a country's happiness ranking?  Let's use the United Nation's World Happiness Rankings country level data to experiment with models that predict happiness rankings well.
Model Key Words (Search categories that describe your model, separated with commas):world happiness, classification, supervised learning
   
Creating your prediction API. (This process may take several minutes.)

[=====================================] Progress: 100% - Complete!                                            

Success! Your Mod

In [ ]:
#Delete Deployment if testing: 

#myplayground.delete_deployment()

Running this function will permanently delete all resources tied to this deployment, 
 including the eval lambda and all models submitted to the model competition.

To confirm, type 'permanently delete':permanently delete


'API deleted successfully.'

# **Part 3: Create a Competition**

In [ ]:
# Create data directory with X_train, y_train_labels, and X_test objects: 
import os
import shutil 

X_train.to_csv("X_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)
y_train.to_csv("y_train.csv", index=False)

os.mkdir("world_happiness_competition_data")
file_list = ["X_train.csv", "X_test.csv", "y_train.csv"]

for file in file_list: 
  shutil.move(file, "world_happiness_competition_data")

In [ ]:
# Create Competition
myplayground.create_competition(data_directory='world_happiness_competition_data', 
                                y_test = y_test,  
                                public=True)


--INPUT COMPETITION DETAILS--

Enter competition name:World Happiness CLassification Competition
Enter competition description:What makes the citizens of one country more happy than the citizens of other countries? Do variables measuring perceptions of corruption, GDP, maintaining a healthy lifestyle, or social support associate with a country's happiness ranking?  Let's use the United Nation's World Happiness Rankings country level data to experiment with models that predict happiness rankings well.

--INPUT DATA DETAILS--

Note: (optional) Save an optional LICENSE.txt file in your competition data directory to make users aware of any restrictions on data sharing/usage.

Enter data description (i.e.- filenames denoting training and test data, file types, and any subfolders where files are stored):Competition data folder contains labeled X_train, y_train, and X_test objects.
Enter optional data license descriptive name (e.g.- 'MIT, Apache 2.0, CC0, Other, etc.'):
Uploading your data. 

In [ ]:
#Instantiate Competition
#--Note: If you start a new session, the first argument should be the Model Playground url in quotes. 
#--e.g.- mycompetition= ai.Competition("https://2121212.execute-api.us-east-1.amazonaws.com/prod/m)
#See Model Playground "Compete" tab for example model submission code.

mycompetition= ai.Competition(myplayground.playground_url)

In [ ]:
#Authorized users can submit new models after setting credentials using modelshare.org username/password
from aimodelshare.aws import set_credentials 

apiurl=myplayground.playground_url # example url from deployed playground: apiurl= "https://123456.execute-api.us-east-1.amazonaws.com/prod/m

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [ ]:
# Optional: Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 1

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1211


Update Runtime model

*Use this function to 1) update the prediction API behind your Model Playground with a new model, chosen from the leaderboard and 2) verify the modelperformance metrics in your Model Playground*

In [ ]:
import aimodelshare as ai
myplayground=ai.ModelPlayground(playground_url="https://3uy3lbiu14.execute-api.us-east-1.amazonaws.com/prod/m")
myplayground.update_runtime_model(model_version=1)

Runtime model & preprocessor for api: https://3uy3lbiu14.execute-api.us-east-1.amazonaws.com/prod/m updated to model version 1.

Model metrics are now updated and verified for this model playground.
